# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy (WFSS) data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [1]:
# Imports
from locals import SourceCatalog
from SEDkit import SED
import astropy.units as q
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in /grp/hst/cdbs/grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:114: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /grp/hst/cdbs/mtab/*_tmg.fits.
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:121: UserWarning: No thermal tables found, no thermal calculations can be performed. No files 

Loading BokehJS ...

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, an `SED` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [2]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the SED object
src = SED(ra=ra, dec=dec, name='LHS 2924', spectral_type='M9', radius=(1.06*q.R_jup,0.05*q.R_jup))

# Find some photometry
src.find_SDSS()
src.find_2MASS()
src.find_WISE()
src.find_PanSTARRS()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting radius to (<Quantity 0.09142448896255928 solRad>, <Quantity 0.017392156215286114 solRad>)
Setting radius to (<Quantity 1.06 jupiterRad>, <Quantity 0.05 jupiterRad>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)

   band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447        21.748 ... 1.1986974328538352e-18   SDSS.u
  SDSS.g  0.4702495279331733        20.958 ...   7.67270104883512e-19   SDSS.g
   PS1.g 0.48491168757054565        20.567 ... 1.3484427632650426e-18    PS1.g
  SDSS.r  0.6175578881059804        18.626 ...  8.821504093946655e-19   SDSS.r
   PS1.r  0.6201185707505671        18.488 ...   6.60097046064012e-19    PS1.r
  Gaia.G  0.6597013110557894       16.6299 ...                    0.0   Gaia.G
  SDSS.i  0.7489976846298569        16.209 ...  2.53988058

Now we can calculate the fundamental parameters and plot the results.

In [3]:
src.results
# 2275K from Fili15

param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,1.06,0.05,jupiterRad
SpT,M9,--,--
SpT_fit,--,--,--
spectral_type,69.0,0.5,--
membership,--,--,--


Let's fit a blackbody for fun.

In [4]:
src.fit_blackbody()


Blackbody fit: 1831 K


In [5]:
fig = src.plot(integral=True)

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

Right now, we are only interested in brown dwarfs so we will include a color cut to only process sources within brown dwarf color space.

In [19]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/fake/')

# Calculate all sources in the pipeline output diretory
cat = SourceCatalog(cat_path, color_cut='brown_dwarfs')

5/22 sources added to catalog after applying 'brown_dwarfs' color cuts


All the sources are stored as `SED` instances (such as the one above) within the `SourceCatalog.sources` attribute.

In [21]:
# Get a source instance
source1 = cat.get_SED(0)

# Calculate the results
print(source1.results)

# Plot it
fig = source1.plot(spectra='all')

    param      value     unc        units    
------------- -------- -------- -------------
         name Source 5       --            --
          age      6.0      4.0           Gyr
     distance       --       --            --
     parallax       --       --            --
       radius       --       --            --
          SpT       --       --            --
      SpT_fit  Opt:M7V       --            --
spectral_type       --       --            --
   membership       --       --            --
         fbol 2.51e-13 6.24e-16 erg / (cm2 s)
         mbol   20.019    0.003            --
         Lbol       --       --            --
     Lbol_sun       --       --            --
         Mbol       --       --            --
         logg       --       --            --
         mass       --       --            --
         Teff       --       --            --
      Teff_bb       --       --            --
     Teff_evo       --       --            --


Hooray! Let's take a look at another one.

In [22]:
# Get the source instance
source2 = cat.get_SED(1)

# Calculate the results
print(source2.results)

# Plot it
fig = source2.plot(spectra='all')

    param      value     unc        units    
------------- -------- -------- -------------
         name Source 7       --            --
          age      6.0      4.0           Gyr
     distance       --       --            --
     parallax       --       --            --
       radius       --       --            --
          SpT       --       --            --
      SpT_fit  Opt:M5V       --            --
spectral_type       --       --            --
   membership       --       --            --
         fbol 1.69e-13 4.04e-16 erg / (cm2 s)
         mbol   20.447    0.003            --
         Lbol       --       --            --
     Lbol_sun       --       --            --
         Mbol       --       --            --
         logg       --       --            --
         mass       --       --            --
         Teff       --       --            --
      Teff_bb       --       --            --
     Teff_evo       --       --            --


Fantastic!

Let's see a table of the results and plot all the SEDs!

In [23]:
cat.results[['name','spectral_type','fbol','radius','Teff_bb','SpT_fit']]

name,spectral_type,fbol,radius,Teff_bb,SpT_fit
,,erg / (cm2 s),solRad,K,
object,object,object,object,object,object
Source 5,None,2.510507681863036e-13,None,None,Opt:M7V
Source 7,None,1.6919413172786906e-13,None,None,Opt:M5V
Source 11,None,8.402079788691585e-13,None,None,Opt:M4V
Source 16,None,2.2711271143675222e-11,None,None,Opt:M4V
Source 17,None,1.2840354911313947e-13,None,None,Opt:M5V


In [15]:
# We imposed a color cut of NIRISS.F150W-NIRISS.F200W in (0.3, 0.5) and NIRISS.F158M-NIRISS.F150W in (-0.2, -0.1)
# Let's see if it worked
color = cat.plot('NIRISS.F150W-NIRISS.F200W', 'NIRISS.F158M-NIRISS.F150W')
show(color)

Nice!